In [1]:
# Read in the inputs and convert them to integers and sort them
inputs = [list(i[:-1]) for i in open("Day11_input.txt").readlines()]

In [2]:
def apply_rule_seat(grid, row, col):
    """
    This function applies the rules to a single seat given a seat layout.
    
    :param grid: list, a list of lists that contains the current seat layout
    :param row: int, the row number
    :param col: int, the column number
    """
    # The rules depend on the current value of the seat
    # If the seat is empty
    if grid[row][col] == 'L':
        # Iterate over the eight surrounding squares
        # Note: i,j = 0,0 occurs in this loop but does not affect the outcome
        for i in range(-1,2):
            for j in range(-1,2):
                # Check the index is valid
                if (0 <= row-i < len(grid)) and (0 <= col-j < len(grid[row])):
                    # If the square is unoccupied we can pass
                    if grid[row-i][col-j] in ("L", "."):
                        pass
                    # Else, it will remain empty if a seat around it is occupied
                    else:
                        return "L"
                # Pass if index is invalid
                else:
                    pass
        # If all surrounding squares are empty, change to occupied
        return "#"
    # If the seat is occupied
    elif grid[row][col] == '#':
        # Initialise a counter to count occupied surrounding seats
        count = 0
        # Iterate over the eight surrounding squares
        # Note: i,j = 0,0 occurs in this loop and will add 1 to the count
        for i in range(-1,2):
            for j in range(-1,2):
                # Check the index is valid
                if (0 <= row-i < len(grid)) and (0 <= col-j < len(grid[row])):
                    # If the square is occupied we add one to the count
                    if grid[row-i][col-j] == "#":
                        count += 1
                    else:
                        pass
                # If the index is invalid, we leave the count alone
                else:
                    pass
        # Once we have finished counting, check if there are 4 + 1 occupied seats
        # surrounding the current seat, noting we've counted ourselves
        if count > 4:
            # The seat is vacated in the count is > 4
            return "L"
        else:
            # Else it remains the same
            return "#"
    # If there is no seat, do nothing
    else:
        return "."

def apply_rule(grid, n=1):
    """
    This function takes a seat layout and applies the rules to all the seats until
    it stabilises.
    
    Note: n is intended to be left empty and counts the iterations
    
    :param grid: list, a list of lists that contains the current seat layout
    :param n: int, the iteration number
    """
    # Copy the supplied seat layout
    new_grid = [line.copy() for line in grid]
    
    # For each row and column in the seat layout
    for row, line in enumerate(grid):
        for col, val in enumerate(line):
            # Apply the rule to get the new seat
            new_grid[row][col] = apply_rule_seat(grid, row, col)
    
    # If the seat layout has stabilied return the layout and n
    if grid == new_grid:
        return new_grid, n
    # Otherwise, repeat the process
    else:
        return apply_rule(new_grid, n+1)

In [3]:
# Run the simulation over the inputs and count the number of occupied seats
stable = apply_rule(inputs)
occupied = sum([sum([val == '#' for val in line]) for line in stable[0]])
# And the solution is...
print(f"After {stable[1]} iterations, there are {occupied:,} seats occupied.")

After 96 iterations, there are 2,441 seats occupied.


In [4]:
def apply_new_rule_seat(grid, row, col):
    """
    This function applies the new rules to a single seat given a seat layout.
    
    :param grid: list, a list of lists that contains the current seat layout
    :param row: int, the row number
    :param col: int, the column number
    """
    # The rules depend on the current value of the seat
    # If the seat is empty
    if grid[row][col] == 'L':
        # Iterate over the eight surrounding squares
        # Note: i,j = 0,0 occurs in this loop but does not affect the outcome
        for i in range(-1,2):
            for j in range(-1,2):
                # We need to search for the nearest seat in each direction
                seat_search = True  # This will indicate if we're still looking for a seat
                k = 1               # This will be the multiplier to check the following seat in the line of sight
                # While we're still searching for the next seat
                while seat_search:
                    # Check the index is valid
                    if (0 <= row-i*k < len(grid)) and (0 <= col-j*k < len(grid[row])):
                        # If the seat is empty, we can spot searching in this direction
                        if grid[row-i*k][col-j*k] == "L":
                            seat_search = False
                        # Else if there is no seat, increment k to look one more seat backwards
                        elif grid[row-i*k][col-j*k] == ".":
                            k += 1
                        # If there is an occupied seat, it will remain empty
                        else:
                            return "L"
                    # Otherwise if the index is invalid, we can stop looking for the next seat
                    else:
                        seat_search = False
        # If no seats around the seat are occupied, seat becomes occupied
        return "#"
    # If the seat is occupied
    elif grid[row][col] == '#':
        # Initialise a counter to count occupied surrounding seats
        count = 0
        # Iterate over the eight surrounding squares
        # Note: i,j = 0,0 occurs in this loop and will add 1 to the count
        for i in range(-1,2):
            for j in range(-1,2):
                # We need to search for the nearest seat in each direction
                seat_search = True  # This will indicate if we're still looking for a seat
                k = 1               # This will be the multiplier to check the following seat in the line of sight
                # While we're still searching for the next seat
                while seat_search:
                    # Check the index is valid
                    if (0 <= row-i*k < len(grid)) and (0 <= col-j*k < len(grid[row])):
                        # If the square is occupied we add one to the count
                        if grid[row-i*k][col-j*k] == "#":
                            count += 1
                            # And we can stop looking for the next seat
                            seat_search = False
                        # If the seat is empty we can stop looking for the next seat
                        elif grid[row-i*k][col-j*k] == "L":
                            seat_search = False
                        # Else if there is no seat, increment k to look one more seat backwards
                        else:
                            k += 1
                    # Otherwise if the index is invalid, we can stop looking for the next seat
                    else:
                        seat_search = False
        # Once we have finished counting, check if there are 5 + 1 occupied seats
        # surrounding the current seat, noting we've counted ourselves
        if count > 5:
            # The seat is vacated in the count is > 5
            return "L"
        else:
            # Else it remains the same
            return "#"
    # If there is no seat, do nothing
    else:
        return "."

def apply_new_rule(grid, n=1):
    """
    This function takes a seat layout and applies the new rules to all the seats
    until it stabilises.
    
    Note: n is intended to be left empty and counts the iterations
    
    :param grid: list, a list of lists that contains the current seat layout
    :param n: int, the iteration number
    """
    # Copy the supplied seat layout
    new_grid = [line.copy() for line in grid]
    
    # For each row and column in the seat layout
    for row, line in enumerate(grid):
        for col, val in enumerate(line):
            # Apply the new rule to get the new seat
            new_grid[row][col] = apply_new_rule_seat(grid, row, col)
    
    # If the seat layout has stabilied return the layout and n
    if grid == new_grid:
        return new_grid, n
    # Otherwise, repeat the process
    else:
        return apply_new_rule(new_grid, n+1)

In [5]:
stable = apply_new_rule(inputs)
occupied = sum([sum([val == '#' for val in line]) for line in stable[0]])
print(f"After {stable[1]} iterations, there are {occupied:,} seats occupied.")

After 89 iterations, there are 2,190 seats occupied.


---